# German Ebay Car Sales 2016

The Data-Set contains offerings of used cars in germany (20 attributes, 371528 examples). These cars where submitted to the website 'ebay Kleinanzeigen' and have been crawled between 2016-03-05 and 2016-04-07.

The dataset can be found here:
https://www.kaggle.com/orgesleka/used-cars-database

### Importing a crap tone of libraries
If you are running this for the first time, you will need to install alot of applications for the libraries that follow. You can run this in your python console, be wary that some may take a while.

In [ ]:
pip install wheel
pip install squarify
pip install pygal
pip install pywaffle
pip install pipwin
pip install plotly
pipwin install numpy
pipwin install pandas
pipwin install shapely
pipwin install gdal
pipwin install fiona
pipwin install pyproj
pipwin install six
pipwin install rtree
pipwin install descartes
pipwin install geopandas

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
from sklearn import preprocessing
import squarify
import geopandas as gp
import shapely
import fiona
import pygal
from pywaffle import Waffle
import plotly.express as px

%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
saved_style_state = matplotlib.rcParams.copy()


### Getting the Files
This is just a small code to pull up the csv file. It will print "File not found!" if the file isnt found in the OpenDataProject Directory


In [ ]:
#Getting the Car data File
filepath = "autos.csv"
cardata= pd.read_csv(filepath)
print("Successfully Loaded Car CSV")

#Getting the Map Files
shp_file_name = "plz-gebiete.shp"
germanburbs = gp.GeoDataFrame.from_file(shp_file_name)
print("Successfully Loaded Map Shape File")

Here is a view of the dataset. It was created using a webscraper on ebay germany. I found an explanation of the columns online as there was a couple weird entries in german, and odd coulumns including 'abtest'

- dateCrawled         : when advert was first crawled, all field-values are taken from this date \n
- name                : headline, which the owner of the car gave to the advert \n
- seller              : 'privat'(ger)/'private'(en) or 'gewerblich'(ger)/'dealer'(en) \n
- offerType           : 'Angebot'(ger)/'offer'(en) or 'Gesuch'(ger)/'request'(en)
- price               : the price on the advert to sell the car
- abtest              : ebay-intern variable (argumentation in discussion-section)
- vehicleType         : one of eight vehicle-categories 
- yearOfRegistration  : at which year the car was first registered
- gearbox             : 'manuell'(ger)/'manual'(en) or 'automatik'(ger)/'automatic'(en)
- powerPS             : the power of the car in PS
- model               : the cars model
- kilometer           : how many kilometres the car has driven
- monthOfRegistration : at which month the car was first registered
- fuelType            : one of seven fuel-categories
- brand               : the cars brand
- notRepairedDamage   : if the car has a damage which is not repaired yet
- dateCreated         : the date for which the advert at 'ebay Kleinanzeigen' was created
- nrOfPictures        : number of pictures in the advert
- postalCode          : where in germany the car is located
- lastSeenOnline      : when the crawler saw this advert last online



In [ ]:
cardata.head(10)

## Simple Car Sales Comparisons

Below are some initial data analysis from simple characteristics provided in the german car data csv

In [ ]:
df = cardata["gearbox"]

man_num = df.str.count("manuell").sum()
aut_num = df.str.count("automatik").sum()

counts = [int(aut_num), int(man_num)]
my_labels = ['Automatic Cars', 'Manual Cars']
plt.pie(counts,labels=my_labels,autopct='%.0f%%',textprops={'fontsize': 25})
plt.title('Car Gearbox Types',fontsize=30)
plt.axis('equal')
fig= plt.figure(figsize=(6,3))
plt.show()

#### So interestingly around 70% of people drive Manual Cars in Germany

In [ ]:
df = cardata["fuelType"]

diesel_num = df.str.count("diesel").sum()
petrol_num = df.str.count("benzin").sum()
lpg_num = df.str.count("lpg").sum()


counts = [int(diesel_num), int(petrol_num), int(lpg_num)]
my_labels = ['Diesel', 'Petrol', 'LPG']
plt.pie(counts,labels=my_labels,autopct='%.0f%%',textprops={'fontsize': 25})
plt.title('Car Fuel Types',fontsize=30)
plt.axis('equal')
fig= plt.figure(figsize=(6,3))
plt.show()

#### And around 70% of cars are petrol run. This dataset does not include electic vehicles, which have increase in production significantly since this data was recorded in 2016

In [ ]:
cardata["yearOfRegistration"][cardata["yearOfRegistration"] >1990 ][cardata["yearOfRegistration"] < 2020].hist()

plt.xticks(fontsize=10, rotation='90')
plt.title("Car Manufacture Year", fontsize=20)
plt.xlabel('\n Year of Manufacture', fontsize=15)
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')



In [ ]:
cardata["price"][cardata["price"] <30000 ].hist()

plt.xticks(fontsize=10, rotation='90')
plt.title("Car Prices", fontsize=20 )
plt.xlabel('\n Price', fontsize=15 )
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')

In [ ]:
cardata["vehicleType"].value_counts().plot(kind="bar")

plt.xticks(fontsize=10, rotation='90')
plt.title("Car Types", fontsize=20 )
plt.xlabel('\n Car Types in German', fontsize=15 )
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')

In [ ]:
e = cardata["model"].value_counts()
e = e.nlargest(20)
e.plot(kind="bar")

plt.xticks(fontsize=10, rotation='90')
plt.title("Car Model Frequency", fontsize=20 )
plt.xlabel('\n Models', fontsize=15 )
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')

In [ ]:
cardata["powerPS"][cardata["powerPS"] <400 ].hist()

plt.xticks(fontsize=10, rotation='90')
plt.title("Car Power", fontsize=20 )
plt.xlabel('\n Car Horsepower', fontsize=15)
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')

In [ ]:
cardata["brand"].value_counts().plot(kind="bar")

plt.xticks(fontsize=10, rotation='90')
plt.title("Car Manufactures", fontsize=20 )
plt.xlabel('\n Manufacturer', fontsize=15)
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')

In [ ]:
Typecounts = cardata['vehicleType'].value_counts().rename_axis('vehicleType').reset_index(name='count')
data = dict(zip(Typecounts['vehicleType'], Typecounts['count']))
fig = plt.figure(FigureClass=Waffle,
                 figsize=(18,8),
                 rows=15,
                 columns=25,
                 values=Typecounts['count'],
                 title={'label': 'Car Types',
                        'loc': 'left',
                        'fontdict': {'fontsize': 20}},
                 labels=["{} ({:,})".format(k, v) for k, v in data.items()],
                 legend={'loc': 'lower left',
                         'bbox_to_anchor': (0, -0.4),
                         'ncol': 6,
                         'framealpha': 0,
                         'fontsize': 12})

Since Volkswagen is the top selling car, i wanted to inspect the top car models that are sold

In [ ]:
df = cardata[cardata["brand"]=='volkswagen']
Typecounts = df['model'].value_counts().rename_axis('model').reset_index(name='count')
data = dict(zip(Typecounts['model'], Typecounts['count']))
fig = plt.figure(FigureClass=Waffle,
                 figsize=(18,8),
                 rows=15,
                 columns=25,
                 values=Typecounts['count'],
                 title={'label': 'Top Volkswagen Car Models',
                        'loc': 'left',
                        'fontdict': {'fontsize': 20}},
                 labels=["{} ({:,})".format(k, v) for k, v in data.items()],
                 legend={'loc': 'lower left',
                         'bbox_to_anchor': (0, -0.4),
                         'ncol': 6,
                         'framealpha': 0,
                         'fontsize': 12})

## Car Brand Analysis

I found the car brands particularly interesting so in this section i filteres the brand data from the original csv and created a new dataset, and made some interesting graphs and comparisons

I am going to make a graph of car sales on mannufactures parent companies. Below is a list of car brands and their corresponding parent companies

- toyota : palexus, daihatsu, toyota
- general motors : chevrolet, buic, cadillac, holden, hsv
- volkswagen : bentley, skoda, audi, lamborghini, bugatti, porshe, volkswagen
- fiat chrysler automobiles : jeep, fiat, dodge, abarth, lancia, alpha_romeo, chrysler
- psa group : peugeot, citreon, opel, vauxhall
- daimler : mercedes_benz, smart, maybach
- bmw : bmw, mini, rolls_royce
- group renault : nissan, infiniti, mitsubishi, renault
- Tata motors : land rover, jaguar
- hyundai : hyundai, kia, genesis
- geely : volvo, lotus
- fuji heavy industries : subaru
- independants : everyone else

In [ ]:
#making some ugly functions
def brandcounter(braand):
    sum = cardata["brand"].str.count(braand).sum()
    return sum

def parentcounterlists(listofbrands):
    brandlist = []
    for i in listofbrands:
        brandlist.append(brandcounter(i))
        sum1 = sum(brandlist)
    return sum1

def parentcounterlistsoflists(listlistofbrands):
    brandlist = []
    for i in listlistofbrands:
        brandlist.append(parentcounterlists(i))
    return brandlist
#parent company and car lists
parent_companies = [
"Toyota", 
"General Motors", 
"Volkswagen",
"Fiat\nChrysler\nAutomobiles", 
"PSA Group", 
"Daimler",
"BMW",
"Group Renault",
"Tata Motors",
"Hyundai",
"Geely", 
"Fuji\nHeavy\nIndustries", 
"Independants" 
]

parent_company_cars = [
["palexus", "daihatsu", "toyota"],
["chevrolet", "buic", "cadillac", "holden", "hsv"],
["bentley", "skoda", "audi", "lamborghini", "bugatti", "porshe", "volkswagen"],
["jeep", "fiat", "dodge", "abarth", "lancia", "alpha_romeo", "chrysler"],
["peugeot", "citroen", "opel", "vauxhall"],
["mercedes_benz", "smart", "maybach"],
["bmw", "mini", "rolls_royce"],
["nissan", "infiniti", "mitsubishi", "renault"],
["land rover", "jaguar"],
["hyundai", "kia", "genesis"],
["volvo", "lotus"],
["subaru"]
]
#Making a new clean dataset 

# This was done by searching brands from parent_company_cars and counting them in the german car data csv

parent_count = list(parentcounterlistsoflists(parent_company_cars))

independant_count = len(cardata["brand"]) - sum(parent_count)
parent_count.append(independant_count)  
    
#Reorganising Data for another Graph
data = cardata["brand"].value_counts()
data1 = data.index.tolist()
data2 = data.tolist()
data3 = []
for i in data1:

    if i in (item for sublist in parent_company_cars for item in sublist):
        f = next(((j, car.index(i))
              for j, car in enumerate(parent_company_cars)
              if i in car),
             None)
        f = f[0]
        data3.append(parent_companies[f])
    else:
        data3.append('Independant') 

brand = pd.DataFrame({'Parent Companies': data3 , 'Quantity of Cars': data2 , 'Car Brand': data1})


I did this very early on in my coding journey and as you can see, is a very unconventional way of reorganising this data. Below is representation of the new dataframe created 'brand'

In [ ]:
brand.sample(10)

## Playing with different plots

Below i explore pretty much the same type of data on a couple of different plots including stacked bar charts, bar charts, treemaps (1D & 2D) and a cool little diagram called a 'Waffle Diagram'

In [ ]:
#This is plotting the data onto a bar graph
df = pd.DataFrame({'Parent Companies': parent_companies , 'Quantity of Cars': parent_count})
df.sort_values('Quantity of Cars',inplace= True, ascending = False)
ax = df.plot.bar(x='Parent Companies',  rot=0, width = 0.5)
plt.xlabel('\n Parent Company', fontsize=15)
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')
plt.title("Car Family Manufactures", fontsize=20 )

I found the representation of this data in the above table quite simplistic so i explored some different graphs including some cool things called Treemaps :)

In [ ]:
# This is a 1D plot onto a treemap
plt.figure(figsize=(20,10))
squarify.plot(sizes=parent_count,
              color=['#221A7C','#202785','#1D348E','#1B4297', '#184FA0', '#165CA9'  , '#1369B2', '#0979B9',"#0089c0", '#0095C2', '#00A2C4', '#00AEC5', \
         '#00BAC7', '#16C0C5','#2BC5C3', '#41CBC0', '#56D0BE', '#6CD6BC', '#77DABB',  '#83DEBA', '#8EE1B8',  '#9AE5B7', \
          '#A5E9B6', '#B0EDB5', '#BCF0B3', '#C7F4B2', '#CCF5B4', '#D1F7B6', '#DCFAB9', '#E6FCBD', '#F0FFC0'],
              label=parent_companies,
              pad=True)

plt.title('Treemap', fontsize=20 )
plt.axis('off');

This treemap is a really simplistic and effective way of representing my data. Although, i want to find a way to further display the subcategories of brands inside the current rectangles. Below is my very first Waffle Diagram

In [ ]:
# This is a Waffle Diagram displaying the exact same information in the treemap above
data = dict(zip(parent_companies, parent_count))
fig = plt.figure(FigureClass=Waffle,
                 figsize=(18,8),
                 rows=15,
                 columns=25,
                 values=parent_count,
                 title={'label': 'Car Manufacturer by Parent Companies',
                        'loc': 'left',
                        'fontdict': {'fontsize': 20}},
                 labels=["{} ({:,})".format(k, v) for k, v in data.items()],
                 legend={'loc': 'lower left',
                         'bbox_to_anchor': (0, -0.4),
                         'ncol': 6,
                         'framealpha': 0,
                         'fontsize': 12})

In [ ]:
# This is another Waffle Diagram, displaying all the car brands
data = dict(zip(data1,data2))
fig = plt.figure(FigureClass=Waffle,
                 figsize=(18,8),
                 rows=15,
                 columns=25,
                 values=data2,
                 title={'label': 'Car Manufacturers',
                        'loc': 'left',
                        'fontdict': {'fontsize': 20}},
                 labels=["{} ({:,})".format(k, v) for k, v in data.items()],
                 legend={'loc': 'lower left',
                         'bbox_to_anchor': (0, -0.4),
                         'ncol': 6,
                         'framealpha': 0,
                         'fontsize': 12})

As you can see, the data is very confusing, especially looking towards the end of the legend as it is very hard to decifer if a certain colour is for example (Is a certain orange plot a bmw, a seat, a hyundai, a porshe or a jaguar?). I researched and found a way to graph 2 dimensions onto the same treemap. The plot is below :)

In [ ]:
fig = px.treemap(brand, 
                 path=['Parent Companies', 'Car Brand'], 
                 values='Quantity of Cars' ,
                #  color='Quantity of Cars'
                )
fig.show()

In [ ]:
df1 = pd.pivot_table(brand, values='Quantity of Cars', index='Parent Companies', columns='Car Brand')
df1.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1.2, 1))
plt.xlabel('\n Parent Company', fontsize=15)
plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')
plt.title("Car Family Manufactures", fontsize=20 )
#rectangle artists


In [ ]:
#I tried sorting it, but it took too long
# sort = brand.groupby('Parent Companies').sum().sort_values('Quantity of Cars', ascending = False).index
# df1 = pd.pivot_table(brand, values='Quantity of Cars', index='Parent Companies', columns='Car Brand').reindex(sort_list) 
# df1.plot(kind='bar', stacked=True).legend(bbox_to_anchor=(1.2, 1))
# plt.xlabel('\n Parent Company', fontsize=15)
# plt.ylabel('Quantity of Cars', fontsize=15, rotation='vertical', ha='right')
# plt.title("Car Family Manufactures", fontsize=20 )

# Maps

I found a map shape file of Germany that included corresponding postal codes that i could link to my car data. I found the shape file here (the site is in German, you might want to translate it):
https://www.suche-postleitzahl.org/downloads

Probably dont run the following cell as it takes a while, but you will see a massive shape file of germany


In [ ]:
germanburbs.geometry.plot()

And below is the dataframe that corresponds with the german shape file. plz is the postalcode.

In [ ]:
germanburbs.sample(10)

## Adding Car data to the Map
Trying to add a count of sales and other information in each postcode. I aim to further add other characteristics like comparing the most prominant car brands regionally

In [ ]:
#Really rough distribution of car sales via postal code
df = cardata["postalCode"]
df.hist()

As you see above this data is quite useless as the geographical data is lost in translation when plotting to a bar graph. All you can really see is that there are more cars in suburbs with a postalcode between 40000-50000

## Sorting through Car Data for a Map Plot
Below is just some ugly functions and lambdas to do some funcky stuff from the cardata.csv and to add it to the dataframe of the German Map datafram (germanburbs).

p.s they are very long and repetitive as i got an error called {'DataFrame' objects are mutable, thus they cannot be hashed}, which stopped me from making a function .

In [ ]:
#This is a function that will run from a lamda that will count the number of cars sold in each postcode
cardata["postalCode"] = cardata["postalCode"].astype(str)
counts = cardata['postalCode'].value_counts().rename_axis('postalCode').reset_index(name='count')
plist = counts['postalCode'].tolist()
def pccounter(pcode):
    pcode = str(pcode)
    if pcode in plist:
        num = counts.loc[counts['postalCode'] == pcode, 'count'].iloc[0]
        return num
    else:
        return 0
#This is the lambda function that adds the count column
germanburbs['count'] = germanburbs.apply(lambda x: pccounter(x['plz']), axis=1)

# I had to write all these down manually as i got the error when trying to make a function that alters a dataframe
# 'DataFrame' objects are mutable, thus they cannot be hashed

#Resorting Data into mean and modes
df = cardata
df['postalCode'] = df['postalCode'].astype(str)

price = df.groupby("postalCode").agg({"price":['mean']}).reset_index()
price['price'] = price['price'].astype(int)
price.columns = price.columns.droplevel(1)

gear = df.groupby("postalCode")['gearbox'].agg(pd.Series.mode)

brand1 = df.groupby("postalCode")['brand'].agg(pd.Series.mode)

fuel = df.groupby("postalCode")['fuelType'].agg(pd.Series.mode)

vtype = df.groupby("postalCode")['vehicleType'].agg(pd.Series.mode)

power = df.groupby("postalCode").agg({"powerPS":['mean']}).reset_index()
power['powerPS'] = power['powerPS'].astype(int)
power.columns = power.columns.droplevel(1)

regoY = df.groupby("postalCode").agg({"yearOfRegistration":['mean']}).reset_index()
regoY['yearOfRegistration'] = regoY['yearOfRegistration'].astype(int)
regoY.columns = regoY.columns.droplevel(1)

regoM = df.groupby("postalCode").agg({"monthOfRegistration":['mean']}).reset_index()
regoM['monthOfRegistration'] = regoM['monthOfRegistration'].astype(int)
regoM.columns = regoM.columns.droplevel(1)

#This is a function that will run from a lamda that will transfer the data above into the map data

counts = cardata['postalCode'].value_counts().rename_axis('postalCode').reset_index(name='count')
plist = counts['postalCode'].tolist()
def pccounter2(pcode,df,column):
    pcode = str(pcode)
    if pcode in plist:
        e = df.loc[df['postalCode'] == pcode,column].iloc[0]
        num = int(e)
        return num
    else:
        return 0
def pcmode(pcode,df1):
    pcode = str(pcode)
    if pcode in plist:
        e = df1[pcode]
        return e
    else:
        return 0

# These are some lambda functions
germanburbs['Gearbox'] = germanburbs.apply(lambda x: pcmode(x['plz'],gear), axis=1)
germanburbs['Car Brand'] = germanburbs.apply(lambda x: pcmode(x['plz'],brand1), axis=1)
germanburbs['Fuel Type'] = germanburbs.apply(lambda x: pcmode(x['plz'],fuel), axis=1)
germanburbs['Vehicle Type'] = germanburbs.apply(lambda x: pcmode(x['plz'],vtype), axis=1)
germanburbs['Car Price Av'] = germanburbs.apply(lambda x: pccounter2(x['plz'],price,'price'), axis=1)
germanburbs['Car Power Av'] = germanburbs.apply(lambda x: pccounter2(x['plz'],power,'powerPS'), axis=1)
germanburbs['Rego Year'] = germanburbs.apply(lambda x: pccounter2(x['plz'],regoY,'yearOfRegistration'), axis=1)
germanburbs['Rego Month'] = germanburbs.apply(lambda x: pccounter2(x['plz'],regoM,'monthOfRegistration'), axis=1)

### Sorting Done
Now From my germanburbs data i am able to tell the avcar price, the most common car type, brand, gearbox etc on any postcode

In [ ]:
germanburbs.sample(10)

## Regional Car Sales 
Below is the plots of regional car sales analysis. There was alot of areas, so i chose to focus around some interesting areas, including capital cities, and the postcode with the most car sales

This is an entire map of the shape file of Germany and the sales numbers in each region.

In [ ]:
df = germanburbs[germanburbs['count']<500]
fig, ax = plt.subplots()
df.plot(ax=ax, column='count', cmap='Reds', legend=True,legend_kwds={'label': "Quantity of Cars for Sale"})
plt.title("Regional Car Sales", fontsize=20 )
plt.xlabel('\n Germany', fontsize=15)
ax.set(
    facecolor='lightblue')


I found this very interesting so i decided to plot the region populations ot see if it related to the number of car sales

Now i am creating (or copying from Bens Pandas doc) some functions allowing me to identify the centroid of postalcode geometry, and then apply it to create some funky plots

In [ ]:
def add_centroid(row):
    return row.geometry.centroid

germanburbs["centroid"] = germanburbs.apply(add_centroid, axis=1)

I wanted to find and inspect the region with the most sales. I found the postcode using the nlargest function and plotted it later on.

In [ ]:
#I found the Purple bit really interesting so i went to inspect it further
x = germanburbs["count"].nlargest(1).index[0]
a = germanburbs.iloc[x]
print(a.centroid)
a.centroid
print(a)


## Mitte - Berlin (Most Car Sales)
The town with the most cars for sale was a town called Mitte in Berlin, postalcode 10115. Interestingly the most common brand in Mitte was not a petrol manual volkswagen, but a Audi Diesel Automatic limosuine

In [ ]:
right_here = shapely.geometry.point.Point(13.3846772483033, 52.53213958289162)
germanburbs["distance_from_berlinmitte"] = burbs.geometry.distance(right_here)
closer_burbs = germanburbs[germanburbs.distance_from_berlinmitte<0.1]
closer_burbs.plot(column='count', cmap='cool', legend=True);

Below is a plot of regions around Mitte-Berlin which was very interesting

In [ ]:
closer_burbs[germanburbs.distance_from_berlinmitte<0.5]
closer_burbs.plot(column='count', cmap='cool', legend=True);

As you can see, i think people who live in berlin put the postal code of mitte down when advertising of selling cars.

## Other Weird Regional Relationships

I explored some other charcteristics of regions using nlargest. I only used postcodes selling 20 cars or more for reliability. 

In [ ]:
#Finding the region with the most powerful cars
x = germanburbs[germanburbs["count"] > 20].nlargest(3,'Car Power Av')
x.head()


In [ ]:
#Finding the region with the least powerful cars
x = germanburbs[germanburbs["count"] > 20].nsmallest(3,'Car Power Av')
x.head()

In [ ]:
#Finding the region with the most expensive cars
x = germanburbs[germanburbs["count"] > 20].nlargest(3,'Car Price Av')
x.head()

In [ ]:
#Finding the regions with the cheapest cars
x = germanburbs[germanburbs["count"] > 20].nsmallest(3,'Car Price Av')
x.head()

# Top City Map plotting

I plotted column relationships on the map of germany and found values for the capital cities of germany. Below is a massive funciton that allows me to do some cool plots later on.

In [ ]:
#Heres some lists and stuff for plotting the capital cities
top_cities = {
    'Berlin': (13.404954, 52.520008), 
    'Cologne': (6.953101, 50.935173),
    'Düsseldorf': (6.782048, 51.227144),
    'Frankfurt am Main': (8.682127, 50.110924),
    'Hamburg': (9.993682, 53.551086),
    'Leipzig': (12.387772, 51.343479),
    'Munich': (11.576124, 48.137154),
    'Dortmund': (7.468554, 51.513400),
    'Stuttgart': (9.181332, 48.777128),
    'Nuremberg': (11.077438, 49.449820),
    'Hannover': (9.73322, 52.37052)
}
top_cities_pc= {
    'Berlin': '10115', 
    'Cologne': '50667',
    'Düsseldorf': '40210',
    'Frankfurt am Main': '60313',
    'Hamburg': '20095',
    'Leipzig': '04103',
    'Munich': '80331',
    'Dortmund': '44135',
    'Stuttgart': '70173',
    'Nuremberg': '90402',
    'Hannover': '30159'
}
Cities_Populations = {
    'Berlin': 3520031, 
    'Cologne': 1060582,
    'Düsseldorf': 612178,
    'Frankfurt am Main': 732688,
    'Hamburg': 1787408,
    'Leipzig': 560472,
    'Munich': 1450381,
    'Dortmund': 586181,
    'Stuttgart': 623738,
    'Nuremberg': 509975,
    'Hannover': 532163
}

#Function for adding column data for the top cities
def cityad(column1):
    table=[]
    pcodes = list(top_cities_pc.values())
    cities = list(top_cities_pc.keys())
    updatedcities = top_cities.copy()
    for i in pcodes:
        df = germanburbs[germanburbs['plz'] == i]
        newV = df[column1].iloc[0]
        table.append(newV)
    count=-1
    for i in cities:
        count = count+1
        value = table[count]
        new_key = str(i)+'\n'+str(value)
        updatedcities[new_key] = updatedcities.pop(str(i))
    return updatedcities


averages = ['count', 'Car Price Av','Car Power Av','Rego Year', 'Rego Month' ]
cat1=['Gearbox', 'Fuel Type']

#This is a very cool function ill use next 
def PPLOT(column1, cmap1):
    if column1 in averages:
        title1 = str(column1 +' by region') #new title
        catagory1 = False
        woo1=None
        if column1 == 'Car Price Av':
            df = germanburbs[germanburbs['Car Price Av']<15000]
        elif column1 == 'Car Power Av':
            df = germanburbs[germanburbs['Car Power Av']<250]
        elif column1 == 'count':
            df = germanburbs[germanburbs['count']<300]
        else:
            df=germanburbs

    elif column1 in cat1:
        germanburbs[column1]=germanburbs[column1].astype(str)
        df1= germanburbs[germanburbs[column1]!="0"] #take out the 0s
        list1 = df1[column1].value_counts().nlargest(2).reset_index() #fetching top 2 results
        list1 = list1['index'].tolist()
        if column1 == 'Fuel Type':
            df = germanburbs[(germanburbs[column1]==list1[0]) + (germanburbs[column1]==list1[1])+ (germanburbs[column1]=='lpg')] #making new dataset slice
        else:
            df = germanburbs[(germanburbs[column1]==list1[0]) + (germanburbs[column1]==list1[1])] #making new dataset slice
        title1 = str("Most used "+ column1 + ' by region') #new title
        catagory1 = True
        
        

    else:
        germanburbs[column1]=germanburbs[column1].astype(str)
        df1 = germanburbs[germanburbs[column1]!="0"] #take out the 0s
        list1 = df1[column1].value_counts().nlargest(5).reset_index() #fetching top 5 results
        list1 = list1['index'].tolist()
        df = germanburbs[(germanburbs[column1]==list1[0]) + (germanburbs[column1]==list1[1])+ (germanburbs[column1]==list1[2]     )+ (germanburbs[column1]==list1[3])+ (germanburbs[column1]==list1[4])] #making new dataset slice
        title1 = str('Top 5 '+ column1 +' by region') #new title
        catagory1 = True
        # woo1 = str('title':column1, 'bbox_to_anchor': (1.35, 0.8))
    
    
    fig, ax = plt.subplots()
    
    plt.xlabel('\n Germany', fontsize=15)

    updatedcities = cityad(column1)
    # Plot cities. 
    for c in updatedcities.keys():
        # Plot city name.
        ax.text(
            x=updatedcities[c][0], 
            # Add small shift to avoid overlap with point.
            y=updatedcities[c][1] + 0.08, 
            s=c, 
            fontsize=8,
            ha='center', 
            c='black',
            weight='bold')
        
        # Plot city location centroid.
        ax.plot(
            updatedcities[c][0], 
            updatedcities[c][1], 
            marker='o',
            c='black', 
            alpha=0.5
            )
    plt.title(str(title1) , fontsize=20 )
    df.plot(ax=ax, column= column1 ,categorical=catagory1, legend=True, cmap=cmap1, alpha=0.9)
    plt.axis('off')
    plt.xlabel('\n Germany', fontsize=15)
    return 


Now i can plot the cities on a map, but i plan to do something abit more with the funciton. Heres what the cities look like though.Below is a graph of the population

In [ ]:
# Importing File
plz_einwohner_df = pd.read_csv(
    'plz_einwohner.csv', 
    sep=',', 
    dtype={'plz': str, 'einwohner': int}
)
# Merge data.
germanburbs = pd.merge(
    left=germanburbs, 
    right=plz_einwohner_df, 
    on='plz',
    how='left'
)
fig, ax = plt.subplots()

germanburbs.plot(
    ax=ax, 
    column='einwohner', 
    categorical=False, 
    legend=True, 
    cmap='autumn_r',
    alpha=0.8
)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )
    
ax.set(
    title='Germany: Population per Region', 
    aspect=1.3, 
    facecolor='lightblue'
);

# My cool function plotter 

I made a big ugly function ot graph all the colums of the germany map dataset, just choose from the column names and have fun :)
This is the column list by the way
- 'count'
- 'Gearbox'
- 'Car Brand'
- 'Fuel Type'
- 'Vehicle Type'
- 'Car Price Av'
- 'Car Power Av'
- 'Rego Year'
- 'Rego Month'

It takes a while, but it is really cool :)

The second varaible is the color map, i suggest you choose from
- 'viridis'
- 'plasma'
- 'inferno'
- 'magma'
- 'gnuplot1'
- 'gnuplot2'
- 'cool'


In [ ]:
#PPLOT(columnname,colormap)
PPLOT('Car Type','plasma')

## Final Thoughts
- 1 - Volkswagan sells alot of cars also owns alot of car manufacturer companies
- 2 - If you live in Germany, there is a high chance you own a 2004 Volkswagen Golf which is worth around 4000 euros, thats around 6000 aud
- 3 - There a more cars for sale in higher popualated areas (wow)
- 4 - If you live in Sternberg, its most likely you drive very quickly (because your car has alot of power), people from DÃsseldorf are slow
- 5 - If you want a cheap car, buy one from Kalbe
- 6 - Leipzig might not use ebay